## 1. Counting missing values
<p>Sports clothing and athleisure attire is a huge industry, worth approximately <a href="https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/">$193 billion in 2021</a> with a strong growth forecast over the next decade! </p>
<p>In this notebook, we play the role of a product analyst for an online sports clothing company. The company is specifically interested in how it can improve revenue. We will dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  </p>
<p>The database provided to us, <code>sports</code>, contains five tables, with <code>product_id</code> being the primary key for all of them: </p>
<h3 id="info"><code>info</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_name</code></td>
<td><code>varchar</code></td>
<td>Name of the product</td>
</tr>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>description</code></td>
<td><code>varchar</code></td>
<td>Description of the product</td>
</tr>
</tbody>
</table>
<h3 id="finance"><code>finance</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>listing_price</code></td>
<td><code>float</code></td>
<td>Listing price for product</td>
</tr>
<tr>
<td><code>sale_price</code></td>
<td><code>float</code></td>
<td>Price of the product when on sale</td>
</tr>
<tr>
<td><code>discount</code></td>
<td><code>float</code></td>
<td>Discount, as a decimal, applied to the sale price</td>
</tr>
<tr>
<td><code>revenue</code></td>
<td><code>float</code></td>
<td>Amount of revenue generated by each product, in US dollars</td>
</tr>
</tbody>
</table>
<h3 id="reviews"><code>reviews</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_name</code></td>
<td><code>varchar</code></td>
<td>Name of the product</td>
</tr>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>rating</code></td>
<td><code>float</code></td>
<td>Product rating, scored from <code>1.0</code> to <code>5.0</code></td>
</tr>
<tr>
<td><code>reviews</code></td>
<td><code>float</code></td>
<td>Number of reviews for the product</td>
</tr>
</tbody>
</table>
<h3 id="traffic"><code>traffic</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>last_visited</code></td>
<td><code>timestamp</code></td>
<td>Date and time the product was last viewed on the website</td>
</tr>
</tbody>
</table>
<h3 id="brands"><code>brands</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>brand</code></td>
<td><code>varchar</code></td>
<td>Brand of the product</td>
</tr>
</tbody>
</table>
<p>We will be dealing with missing data as well as numeric, string, and timestamp data types to draw insights about the products in the online store. Let's start by finding out how complete the data is.</p>

In [139]:
%%sql
postgresql:///sports

SELECT count(*) as total_rows,
    count(i.description) as count_description,
    count(f.listing_price) as count_listing_price,
    count(t.last_visited) as count_last_visited
FROM info i 
JOIN finance f
ON i.product_id = f.product_id 
JOIN traffic t
ON i.product_id = t.product_id;

1 rows affected.


total_rows,count_description,count_listing_price,count_last_visited
3179,3117,3120,2928


In [140]:
%%nose

last_output = _
last_output_df = last_output.DataFrame()

def test_columns():
    assert "total_rows" in set(last_output_df.columns), \
    """Did you alias the count of all products as "total_rows"?"""
    assert set(last_output_df.columns) == set(['total_rows', 'count_description', 'count_listing_price',
           'count_last_visited']), \
    """Did you select four columns and use the aliases in the instructions?"""

def test_shape():
    assert last_output_df.shape[0] == 1, \
    """Did you return a single row containing the count of values for each column?"""
    assert last_output_df.shape[1] == 4, \
    """Did you select four columns?"""

def test_values():
    assert last_output_df.values.tolist() == [[3179, 3117, 3120, 2928]], \
    """Did you correctly calculate the values for each column? Expected different results."""

3/3 tests passed


## 2. Nike vs Adidas pricing
<p>We can see the database contains 3,179 products in total. Of the columns we previewed, only one &mdash; <code>last_visited</code> &mdash; is missing more than five percent of its values. Now let's turn our attention to pricing. </p>
<p>How do the price points of Nike and Adidas products differ? Answering this question can help us build a picture of the company's stock range and customer market. We will run a query to produce a distribution of the <code>listing_price</code> and the count for each price, grouped by <code>brand</code>. </p>

In [141]:
%%sql

SELECT b.brand, 
       CAST(listing_price AS Integer) as listing_price, 
       COUNT(f.*)
FROM brands b
JOIN finance f
ON b.product_id = f.product_id
WHERE f.listing_price > 0
GROUP BY b.brand, listing_price
ORDER BY listing_price DESC;

 * postgresql:///sports
77 rows affected.


brand,listing_price,count
Adidas,300,2
Adidas,280,4
Adidas,240,5
Adidas,230,8
Adidas,220,11
Nike,200,1
Adidas,200,8
Nike,190,2
Adidas,190,7
Nike,180,4


In [142]:
%%nose

last_output = _
last_output_df = last_output.DataFrame()

def test_columns():
    assert set(last_output_df.columns) == set(['brand', 'count', 'listing_price']), \
    """Did you select the correct columns and alias the first as "total_rows"?"""

def test_shape():
    assert last_output_df.shape[0] == 77, \
    """Did you correctly aggregate by brand? Expected the output to contain 77 products."""
    assert last_output_df.shape[1] == 3, \
    """The output should contain three columns: "brand", "listing_price", and "count"?"""

def test_values():
    assert last_output_df.iloc[0].values.tolist() == ['Adidas', 300, 2], \
    """Did you sort the results by "listing_price" in descending order?"""

3/3 tests passed


## 3. Labeling price ranges
<p>It turns out there are 77 unique prices for the products in our database, which makes the output of our last query quite difficult to analyze. </p>
<p>Let's build on our previous query by assigning labels to different price ranges, grouping by <code>brand</code> and <code>label</code>. We will also include the total <code>revenue</code> for each price range and <code>brand</code>. </p>

In [143]:
%%sql

SELECT b.brand,
        COUNT(f.*),
        SUM(f.revenue) as total_revenue,
        CASE WHEN f.listing_price < 42 THEN 'Budget'
             WHEN f.listing_price >= 42 AND f.listing_price < 74 THEN 'Average'
             WHEN f.listing_price >= 74 AND f.listing_price < 129 THEN 'Expensive'
            ELSE 'Elite' 
        END AS price_category
FROM brands b
JOIN finance f
ON b.product_id = f.product_id
WHERE b.brand IS NOT NULL
GROUP BY b.brand, price_category
ORDER BY total_revenue DESC;

 * postgresql:///sports
8 rows affected.


brand,count,total_revenue,price_category
Adidas,849,4626980.069999999,Expensive
Adidas,1060,3233661.060000001,Average
Adidas,307,3014316.8299999987,Elite
Adidas,359,651661.1200000002,Budget
Nike,357,595341.0199999992,Budget
Nike,82,128475.59000000003,Elite
Nike,90,71843.15000000004,Expensive
Nike,16,6623.5,Average


In [144]:
%%nose

last_output = _
last_output_df = last_output.DataFrame()

def test_columns():
    assert set(last_output_df.columns) == set(['brand', 'price_category', 'count', 'total_revenue']), \
    """Did you select the correct columns? Expected "brand", "price_category", "count", and "total_revenue"."""
    
def test_shape():
    assert last_output_df.shape[0] == 8, \
    "Did you group by brand and labels? Expected there to be eight rows."
    assert last_output_df.shape[1] == 4, \
    "Did you select four columns?"
    
def test_values():
    assert last_output_df[:4].values.tolist() == [['Adidas', 849, 4626980.069999999, 'Expensive'],
     ['Adidas', 1060, 3233661.060000001, 'Average'],
     ['Adidas', 307, 3014316.8299999987, 'Elite'],
     ['Adidas', 359, 651661.1200000002, 'Budget']], \
    "Did you correctly calculate values for Adidas products? Expected something different."
    assert last_output_df[4:].values.tolist() == [['Nike', 357, 595341.0199999992, 'Budget'],
     ['Nike', 82, 128475.59000000003, 'Elite'],
     ['Nike', 90, 71843.15000000004, 'Expensive'],
     ['Nike', 16, 6623.5, 'Average']], \
    "Did you correctly calculate values for Nike products? Expected something different."

3/3 tests passed


## 4. Average discount by brand
<p>Interestingly, grouping products by brand and price range allows us to see that Adidas items generate more total revenue regardless of price category! Specifically, <code>"Elite"</code> Adidas products priced \$129 or more typically generate the highest revenue, so the company can potentially increase revenue by shifting their stock to have a larger proportion of these products!</p>
<p>Note we have been looking at <code>listing_price</code> so far. The <code>listing_price</code> may not be the price that the product is ultimately sold for. To understand <code>revenue</code> better, let's take a look at the <code>discount</code>, which is the percent reduction in the <code>listing_price</code> when the product is actually sold. We would like to know whether there is a difference in the amount of <code>discount</code> offered between brands, as this could be influencing <code>revenue</code>.</p>

In [145]:
%%sql
postgresql:///sports
    
SELECT b.brand,
       AVG(discount)*100 as average_discount
FROM brands b
JOIN finance f
ON b.product_id = f.product_id
WHERE b.brand IS NOT NULL
GROUP BY b.brand;

2 rows affected.


brand,average_discount
Nike,0.0
Adidas,33.452427184465606


In [146]:
%%nose

last_output = _
last_output_df = last_output.DataFrame()

def test_columns():
    assert set(last_output_df.columns) == set(['brand', 'average_discount']), \
    """Did you select the correct columns? Expected "brand" and "average_discount"."""
    
def test_shape():
    assert last_output_df.shape[0] == 2, \
    "Did you group by brand? Expected two rows, one per brand."
    assert last_output_df.shape[1] == 2, \
    "Did you select two columns?"
    
def test_values():
    assert last_output_df.iloc[:, 1].values.tolist() == [0.0, 33.452427184465606], \
    "Did you correctly calculate the average discount for the two brands?"

3/3 tests passed


## 5. Correlation between revenue and reviews
<p>Strangely, no <code>discount</code> is offered on Nike products! In comparison, not only do Adidas products generate the most revenue, but these products are also heavily discounted! </p>
<p>To improve revenue further, the company could try to reduce the amount of discount offered on Adidas products, and monitor sales volume to see if it remains stable. Alternatively, it could try offering a small discount on Nike products. This would reduce average revenue for these products, but may increase revenue overall if there is an increase in the volume of Nike products sold. </p>
<p>Now explore whether relationships exist between the columns in our database. We will check the strength and direction of a correlation between <code>revenue</code> and <code>reviews</code>. </p>

In [147]:
%%sql

SELECT CORR(f.revenue, r.reviews) AS review_revenue_corr
FROM finance f
JOIN reviews r
ON f.product_id = r.product_id;

 * postgresql:///sports
1 rows affected.


review_revenue_corr
0.6518512283481301


In [148]:
%%nose

last_output = _
last_output_df = last_output.DataFrame()

def test_columns():
    assert set(last_output_df.columns) == set(['review_revenue_corr']), \
    """Did you calculate the correlation between reviews and revenue, aliasing as "review_revenue_corr"?"""
    
def test_shape():
    assert last_output_df.shape == (1, 1), \
    "Did you calculate the correlation between reviews and revenue?"
    
def test_values():
    assert last_output_df.values.tolist() == [[0.6518512283481301]], \
    "Did you correctly calculate how reviews correlates with revenue?"

3/3 tests passed


## 6. Ratings and reviews by product description length
<p>Interestingly, there is a strong positive correlation between <code>revenue</code> and <code>reviews</code>. This means, potentially, if we can get more reviews on the company's website, it may increase sales of those items with a larger number of reviews. </p>
<p>Perhaps the length of a product's <code>description</code> might influence a product's <code>rating</code> and <code>reviews</code> &mdash; if so, the company can produce content guidelines for listing products on their website and test if this influences <code>revenue</code>. Let's check this out!</p>

In [149]:
%%sql

SELECT TRUNC(Length(description)/100.0) *100 as description_length,
       ROUND(AVG(CAST(rating AS numeric)),2) as  average_rating
FROM info i
JOIN reviews r 
ON i.product_id = r.product_id
WHERE description IS NOT NULL
GROUP BY description_length
ORDER BY description_length;


 * postgresql:///sports
7 rows affected.


description_length,average_rating
0,1.87
100,3.21
200,3.27
300,3.29
400,3.32
500,3.12
600,3.65


In [150]:
%%nose

last_output = _
last_output_df = last_output.DataFrame()

def test_columns():
    assert set(last_output_df.columns) == set(['description_length', 'average_rating']), \
    """Did you select the correct columns use the aliases "description_length" and "average_rating"?"""
    
def test_shape():
    assert last_output_df.shape[0] == 7, \
    """Did you create bins of 100 characters for "description_length"? Expected the output to contain seven rows."""
    assert last_output_df.shape[1] == 2, \
    "Expected the output to contain two columns."
    
def test_values():
    last_output_df = last_output.DataFrame().values.astype("float")
    assert last_output_df[0].tolist() == [0.0, 1.87], \
    """Did you sort the results by "description_length" in ascending order?"""
    assert last_output_df[-1].tolist() == [600.0, 3.65], \
    "Did you correctly calculate the results? Expected a different average rating for the largest description length bin."

3/3 tests passed


## 7. Reviews by month and brand
<p>Unfortunately, there doesn't appear to be a clear pattern between the length of a product's <code>description</code> and its <code>rating</code>.</p>
<p>As we know a correlation exists between <code>reviews</code> and <code>revenue</code>, one approach the company could take is to run experiments with different sales processes encouraging more reviews from customers about their purchases, such as by offering a small discount on future purchases. </p>
<p>Let's take a look at the volume of <code>reviews</code> by month to see if there are any trends or gaps we can look to exploit.</p>

In [151]:
%%sql

SELECT b.brand, 
       DATE_PART('month', last_visited) as month,
       COUNT(r.*) as num_reviews
FROM brands b
JOIN traffic t
ON b.product_id = t.product_id
JOIN reviews r
ON r.product_id = t.product_id
WHERE b.brand IS NOT NULL
GROUP BY 1,2 
HAVING DATE_PART('month', last_visited) IS NOT NULL
ORDER BY 1,2;

 * postgresql:///sports
24 rows affected.


brand,month,num_reviews
Adidas,1.0,253
Adidas,2.0,272
Adidas,3.0,269
Adidas,4.0,180
Adidas,5.0,172
Adidas,6.0,159
Adidas,7.0,170
Adidas,8.0,189
Adidas,9.0,181
Adidas,10.0,192


## Top Revenue Generated Products with Brands  

In [152]:
%%sql

WITH highest_revenue_product AS
(  
   SELECT i.product_name,
          b.brand,
          revenue
   FROM finance f
   JOIN info i
   ON f.product_id = i.product_id
   JOIN brands b
   ON b.product_id = i.product_id
   WHERE product_name IS NOT NULL 
     AND revenue IS NOT NULL 
     AND brand IS NOT NULL
)
SELECT product_name,
       brand,
       revenue,
        RANK() OVER (ORDER BY revenue DESC) AS product_rank
FROM highest_revenue_product
LIMIT 10;

 * postgresql:///sports
10 rows affected.


product_name,brand,revenue,product_rank
Air Jordan 10 Retro,Nike,64203.93,1
Unisex Originals CRAIG GREEN KONTUUR II SHOES,Adidas,37150.45,2
Unisex Originals CRAIG GREEN KONTUUR I SHOES,Adidas,34990.54,3
Men's adidas Running Universal Works Ultraboost 19 Shoes,Adidas,33838.31,4
Men's adidas Originals ZX 4000 4D Shoes,Adidas,31246.88,5
Men's Running Ultraboost PB Shoes,Adidas,30454.31,6
Women's Running Ultraboost 20 Shoes,Adidas,29986.24,7
UNISEX adidas Originals Pharrell Williams Hu NMD Proud Shoes,Adidas,29698.65,8
Women's adidas by Stella Mccartney Running UltraBoost 20 Shoes,Adidas,28834.4,9
Men's Originals Salvapor 032c Shoes,Adidas,28762.31,10


#### Highest revenue generated by Nike brand which is around $64k, when closly look into top revenue products, the shoes product generating highest revenue. Let's check the footwear product performance.    

In [153]:
%%nose

last_output = _
last_output_df = last_output.DataFrame()

def test_columns():
    assert set(last_output_df.columns) == set(['brand', 'month', 'num_reviews']), \
    """Did you select the correct columns? Expected "brand", "month", and "num_reviews"."""
    
def test_shape():
    assert last_output_df.shape[0] == 24, \
    "Did you group by brand and month?"
    assert last_output_df.shape[1] == 3, \
    "Did you select three columns?"
    
def test_values():
    assert last_output_df.iloc[0].values.tolist() == ['Adidas', 1.0, 253], \
    "Expected the first row to contain the number of reviews for Adidas products in January."
    assert last_output_df.iloc[-1].values.tolist() == ['Nike', 12.0, 35.0], \
    "Expected the last row to contain the number of reviews for Nike products in December."
    assert max(last_output_df["num_reviews"]) == 272, \
    "Did you correctly calculate the number of reviews? Expected the largest number of reviews to be 272 for Adidas products in February."

0/3 tests passed; 3 failed
__main__.test_columns
Traceback (most recent call last):
  File "/usr/lib/python3.9/unittest/case.py", line 59, in testPartExecutor
    yield
  File "/usr/lib/python3.9/unittest/case.py", line 592, in run
    self._callTestMethod(testMethod)
  File "/usr/lib/python3.9/unittest/case.py", line 550, in _callTestMethod
    method()
  File "/usr/local/lib/python3.9/dist-packages/nose/case.py", line 198, in runTest
    self.test(*self.arg)
  File "<string>", line 6, in test_columns
AssertionError: Did you select the correct columns? Expected "brand", "month", and "num_reviews".

__main__.test_shape
Traceback (most recent call last):
  File "/usr/lib/python3.9/unittest/case.py", line 59, in testPartExecutor
    yield
  File "/usr/lib/python3.9/unittest/case.py", line 592, in run
    self._callTestMethod(testMethod)
  File "/usr/lib/python3.9/unittest/case.py", line 550, in _callTestMethod
    method()
  File "/usr/local/lib/python3.9/dist-packages/nose/case.py", lin

## 8. Footwear product performance
<p>Looks like product reviews are highest in the first quarter of the calendar year, so there is scope to run experiments aiming to increase the volume of reviews in the other nine months!</p>
<p>So far, we have been primarily analyzing Adidas vs Nike products. Now, let's switch our attention to the type of products being sold. As there are no labels for product type, we will create a Common Table Expression (CTE) that filters <code>description</code> for keywords, then use the results to find out how much of the company's stock consists of footwear products and the median <code>revenue</code> generated by these items.</p>

In [154]:
%%sql

with footwear AS 
( SELECT i.description, 
         f.revenue
  FROM info i
  INNER JOIN finance f
  ON i.product_id = f.product_id
  WHERE i.description ILIKE '%shoe%' 
        OR i.description ILIKE '%trainer%' 
        OR i.description ILIKE '%foot%' 
        AND i.description IS NOT NULL 
)
select COUNT(*) as num_footwear_products,
       percentile_disc(0.5) WITHIN GROUP(ORDER BY revenue) as median_footwear_revenue
FROM footwear;

 * postgresql:///sports
1 rows affected.


num_footwear_products,median_footwear_revenue
2700,3118.36


In [155]:
%%nose

last_output = _
last_output_df = last_output.DataFrame()

def test_columns():
    assert set(last_output_df.columns) == set(['num_footwear_products', 'median_footwear_revenue']), \
    "Did you select the correct columns and use the aliases specified in the instructions?"
    
def test_shape():
    assert last_output_df.shape[0] == 1, \
    "Expected the output to contain one row."
    assert last_output_df.shape[1] == 2, \
    "Expected the output to contain two columns."
    
def test_values():
    assert last_output_df.iloc[0,0] == 2700, \
    "Did you count the number of footwear products?"
    assert last_output_df.iloc[0,1] == 3118.36, \
    "Did you calculate the median revenue for footwear products?"

3/3 tests passed


## 9. Clothing product performance
<p>Recall from the first task that we found there are 3,117 products without missing values for <code>description</code>. Of those, 2,700 are footwear products, which accounts for around 85% of the company's stock. They also generate a median revenue of over $3000 dollars!</p>
<p>This is interesting, but we have no point of reference for whether footwear's <code>median_revenue</code> is good or bad compared to other products. So, for our final task, let's examine how this differs to clothing products. We will re-use <code>footwear</code>, adding a filter afterward to count the number of products and <code>median_revenue</code> of products that are not in <code>footwear</code>.</p>

In [156]:
%%sql

with footwear AS 
( SELECT i.description, 
         f.revenue
  FROM info i
  INNER JOIN finance f
  ON i.product_id = f.product_id
  WHERE i.description ILIKE '%shoe%' 
        OR i.description ILIKE '%trainer%' 
        OR i.description ILIKE '%foot%' 
        AND i.description IS NOT NULL 
)
select COUNT(i.*) as num_clothing_products,
       percentile_disc(0.5) WITHIN GROUP(ORDER BY revenue) as median_clothing_revenue
FROM info i
INNER JOIN finance f
ON i.product_id = f.product_id
WHERE i.description NOT IN (select description from footwear);


 * postgresql:///sports
1 rows affected.


num_clothing_products,median_clothing_revenue
417,503.82


In [157]:
%%nose

last_output = _
last_output_df = last_output.DataFrame()

def test_columns():
    assert set(last_output_df.columns) == set(['num_clothing_products', 'median_clothing_revenue']), \
    "Did you select the correct columns and use the aliases specified in the instructions?"
    
def test_shape():
    assert last_output_df.shape[0] == 1, \
    "Expected the output to contain one row."
    assert last_output_df.shape[1] == 2, \
    "Expected the output to contain two columns."
    
def test_values():
    assert last_output_df.iloc[0,0] == 417, \
    "Did you count the number of clothing products? Expected there to be 417 items."
    assert last_output_df.iloc[0,1] == 503.82, \
    "Did you calculate the median revenue for clothing products? Expected it to be $503.82."

3/3 tests passed
